In [5]:
from src.scraping.get_data import *
from src.utils.checks import *
from src.portfolio.data_manipulation import *
from src.plots.basics import *
from src.computations.compute_returns import *
from src.computations.metrics import *
from src.utils.basics import *
from src.plots.basics_streamlit import *

get_data_info()
data = get_data_updated_2025()

In [2]:
download_data_updated()

File downloaded and renamed to MSCI_World.csv
File downloaded and renamed to MSCI_Europe_Small_Cap_Value_Weighted.csv
File downloaded and renamed to MSCI_Europe.csv
File downloaded and renamed to MSCI_ACWI.csv
File downloaded and renamed to S&P_500_Minimum_Volatility.csv
File downloaded and renamed to MSCI_Emerging_Markets.csv
File downloaded and renamed to FTSE_World_Government_Bond_Developed_Markets.csv
File downloaded and renamed to S&P_500.csv
File downloaded and renamed to MSCI_USA_Small_Cap_Value_Weighted.csv
File downloaded and renamed to MSCI_World_Momentum.csv
File downloaded and renamed to MSCI_World_Quality.csv


In [6]:
data

,Date,MSCI Europe Small Cap Value Weighted,MSCI USA Small Cap Value Weighted.1,MSCI Europe,MSCI World Momentum,MSCI ACWI,MSCI World,S&P 500 Minimum Volatility,MSCI Emerging Markets,FTSE World Government Bond - Developed Markets,S&P 500,MSCI USA Small Cap Value Weighted,MSCI World Sector Neutral Quality
0,01/1975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,02/1975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,03/1975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04/1975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,05/1975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,03/2025,149258.620455,301474.781370,740100.039015,253014.321457,225203.342503,935684.079624,87854.767453,341488.238952,56062.489396,302887.181519,301474.781370,72618.131411
603,04/2025,151103.730645,274195.338702,734564.922807,248540.215643,215995.891590,897683.562553,82429.995779,328764.402587,55177.416831,286073.221001,274195.338702,69198.447553
604,05/2025,161707.718540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72460.238942
605,06/2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
get_list_of_portfolios()

['100_2factors',
 '80_20_ACWI',
 '80_20_World',
 '80_20_2factors',
 '100_1factor',
 'lifestrategy60',
 'lifestrategy80',
 'lifestrategy20',
 'lifestrategy0',
 'lifestrategy40',
 'lifestrategy100',
 '80_20_1factor',
 '60_40_World']

In [5]:
PORTFOLIOS = [
    '100_2factors',
    '80_20_2factors',
    '80_20_1factor',
    '80_20_ACWI',
    '80_20_World'
]

PORTOFOLIO_NAMES = {
    '100_2factors'  : '100% Stocks + SmallCap Value + Momentum',
    '80_20_2factors': '80% Stocks + 20% Bonds + SmallCap Value + Momentum',
    '80_20_1factor' : '80% Stocks + 20% Bonds + SmallCap Value',
    '80_20_ACWI'    : '80% Stocks(ACWI) + 20% Bonds',
    '80_20_World'   : '80% Stocks(World) + 20% Bonds',
}

PORTFOLIO_DESCRIPTIONS = {
    '100_2factors'  : 'This portfolio invests 100% in stocks, focusing on small-cap value and momentum strategies.',
    '80_20_2factors': 'This portfolio allocates 80% to stocks and 20% to bonds, incorporating small-cap value and momentum factors.',
    '80_20_1factor' : 'This portfolio invests 80% in stocks and 20% in bonds, focusing on small-cap value.',
    '80_20_ACWI'    : 'This portfolio invests 80% in global stocks (ACWI) and 20% in bonds.',
    '80_20_World'   : 'This portfolio invests 80% in world stocks and 20% in bonds.'
}

# Add another group of portfolios in col2
SIMPLE_PORTFOLIOS = [
    'lifestrategy100',
    'lifestrategy80',
    'lifestrategy60',
    'lifestrategy40',
    'lifestrategy20',
    'lifestrategy0'
]

SIMPLE_PORTFOLIO_NAMES = {
    'lifestrategy100': '100% Stocks',
    'lifestrategy80' : '80% Stocks + 20% Bonds',
    'lifestrategy60' : '60% Stocks + 40% Bonds',
    'lifestrategy40' : '40% Stocks + 60% Bonds',
    'lifestrategy20' : '20% Stocks + 80% Bonds',
    'lifestrategy0'  : '100% Bonds'
}

SIMPLE_PORTFOLIO_DESCRIPTIONS = {
    'lifestrategy100': 'This portfolio invests 100% in stocks, suitable for aggressive strategies seeking high growth. Note that stocks are MSCI World',
    'lifestrategy80' : 'This portfolio allocates 80% to stocks and 20% to bonds, balancing growth and stability. Stocks are MSCI World',
    'lifestrategy60' : 'This portfolio invests 60% in stocks and 40% in bonds, providing moderate growth with some stability. Stocks are MSCI World',
    'lifestrategy40' : 'This portfolio invests 40% in stocks and 60% in bonds, focusing on stability with some growth potential. Stocks are MSCI World',
    'lifestrategy20' : 'This portfolio invests 20% in stocks and 80% in bonds, prioritizing stability with minimal growth. Stocks are MSCI World',
    'lifestrategy0'  : 'This portfolio invests 100% in bonds, suitable for conservative strategies seeking capital preservation.'
}

ALL_PORTFOLIOS = PORTFOLIOS + SIMPLE_PORTFOLIOS
ALL_PORTFOLIOS

['100_2factors',
 '80_20_2factors',
 '80_20_1factor',
 '80_20_ACWI',
 '80_20_World',
 'lifestrategy100',
 'lifestrategy80',
 'lifestrategy60',
 'lifestrategy40',
 'lifestrategy20',
 'lifestrategy0']

In [6]:
DEFAULT_LAYOUT = {
    "template": "simple_white",
    "width": 1200,
    "height": 800,
    "legend_title": "Portfolio",
    "title_font_size": 25,
    "title_font_family": "Arial",
    "legend": dict(font=dict(size=22)),
    "xaxis": dict(
        tickformat="%Y",
        title_font=dict(size=22, color="black", family="Arial"),
        tickfont=dict(size=16, color="black"),
        linecolor="black",
        linewidth=2,
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=1
    ),
    "yaxis": dict(
        title_font=dict(size=22, color="black", family="Arial"),
        tickfont=dict(size=16, color="black"),
        linecolor="black",
        tickformat=".1f",           # Show one decimal
        tickprefix="",              # No prefix needed; suffix adds %
        ticksuffix="%",            # ✅ Add percent sign without multiplying
        linewidth=2,
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=1
    ),
    "font": dict(color="black")
}

In [7]:
def plot_return_distributions_streamlit(df):
    """
    Create an interactive violin plot showing portfolio return distributions,
    with custom median and 5th percentile lines using Plotly and matplotlib colors.
    """
    def _prepare_data(df):
        returns_df = df.dropna(how='all')
        long_df = returns_df.melt(id_vars='Date', var_name='Portfolio', value_name='Return').dropna()
        return long_df

    def _get_colors(portfolios):
        cmap = plt.get_cmap('tab10')
        return {
            p: '#{0:02x}{1:02x}{2:02x}'.format(
                int(255 * r), int(255 * g), int(255 * b)
            )
            for p, (r, g, b, _) in zip(portfolios, cmap(range(len(portfolios))))
        }

    def _get_plot_shapes_and_annotations(medians, p5s, y_positions, colors, line_height=0.6):
        shapes, annotations = [], []
        for portfolio, y_pos in y_positions.items():
            median_val = medians[portfolio]
            p5_val = p5s[portfolio]

            # Median line
            shapes.append(dict(
                type='line', x0=median_val, x1=median_val,
                y0=y_pos - line_height/2, y1=y_pos + line_height/2,
                line=dict(color=colors[portfolio], width=3),
                opacity=1,
                xref='x', yref='y'
            ))
            annotations.append(dict(
                x=median_val, y=y_pos + line_height / 2 + 0.1,
                text=f"Median: {median_val:.2%}",
                showarrow=False,
                font=dict(color=colors[portfolio], size=16),
                xanchor='left'
            ))

            # 5th percentile line
            shapes.append(dict(
                type='line', x0=p5_val, x1=p5_val,
                y0=y_pos - line_height/2, y1=y_pos + line_height/2,
                line=dict(color=colors[portfolio], width=3, dash='dash'),
                opacity=1,
                xref='x', yref='y'
            ))
            annotations.append(dict(
                x=p5_val, y=y_pos - line_height / 2 + 0.65,
                text=f"5th %: {p5_val:.2%}",
                showarrow=False,
                font=dict(color=colors[portfolio], size=16),
                xanchor='left'
            ))

        return shapes, annotations

    # Data preparation
    long_df = _prepare_data(df)
    portfolios = long_df['Portfolio'].unique()
    colors = _get_colors(portfolios)
    plot_height = 230 * len(portfolios)

    # Seaborn style (optional)
    sns.set(style="whitegrid", context="talk")

    # Violin plot
    fig = px.violin(
        long_df,
        y='Portfolio',
        x='Return',
        color='Portfolio',
        box=False,
        points='all',
        color_discrete_map=colors,
        template='simple_white'
    )

    # Order & positioning
    category_order = fig.layout.yaxis.categoryarray or portfolios.tolist()
    y_positions = {p: i for i, p in enumerate(category_order)}

    # Statistics
    medians = long_df.groupby('Portfolio')['Return'].median()
    p5s = long_df.groupby('Portfolio')['Return'].quantile(0.05)
    shapes, annotations = _get_plot_shapes_and_annotations(medians, p5s, y_positions, colors)

    # Final layout update
    fig.update_layout(
        height=plot_height,
        width=1200,
        violingap=0.25,
        violinmode='overlay',
        shapes=shapes,
        annotations=annotations,
        template='simple_white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.05,
            xanchor="center",
            x=0.5,
            font=dict(size=22)
        ),
        font=dict(color="black"),
        title_font_size=25,
        title_font_family="Arial",
        title = " ",
        xaxis=dict(
            title="Annualized Return (%)",
            title_font=dict(size=22, color="black", family="Arial"),
            tickfont=dict(size=16, color="black"),
            tickformat=".1%",
            linecolor="black",
            linewidth=2,
            showgrid=True,
            gridcolor="lightgray"
        ),
        yaxis=dict(
            title="Portfolio",
            title_font=dict(size=22, color="black", family="Arial"),
            tickfont=dict(size=16, color="black"),
            linecolor="black",
            linewidth=2,
            showgrid=True,
            gridcolor="lightgray",
            categoryorder='array',
            categoryarray=category_order
        )
    )
        # Customize hover template for each trace
    fig.update_traces(
        hoveron = 'points+kde'
        )

    return fig


In [ ]:
portfolios = ['100_2factors',
              '80_20_2factors',
              '80_20_1factor',
              '80_20_ACWI',
              '80_20_World']

portfolios = [ 'lifestrategy40',
               'lifestrategy20',
               'lifestrategy0']

for years in [5]:
    final_results = None 
    for portfolio_name in portfolios:

        portfolio = load_portfolio(portfolio_name)
        data, weights = prune_data_portfolio(portfolio)

        # Compute the returns
        results = compute_portfolio_returns_combined(*prune_data_portfolio(portfolio), years=years)
        results = results.rename(columns={results.columns[-1]: portfolio_name})
        results[portfolio_name] = results[portfolio_name].round(5)

        final_results = merge_results(final_results, results)


    final_results_totalreturn = final_results.copy()

    for portfolio_name in portfolios:    
        final_results_totalreturn[portfolio_name] = (final_results[portfolio_name] + 1)**years - 1


In [28]:
def plot_total_returns_streamlit(results, years=None):
    """
    Plots the total returns of indices over time interactively using Plotly and Seaborn style.

    Parameters:
        results (dict or DataFrame): The results data containing 'Date' and portfolio return columns.
        years (int, optional): Number of years used for rolling window annotation.
    """
    df = pd.DataFrame(results)
    df["Date"] = pd.to_datetime(df["Date"], format="%m/%Y")
    df.set_index("Date", inplace=True)

    sns.set(style="whitegrid", context="talk")

    df_reset = df.reset_index()
    df_melt = df_reset.melt(id_vars="Date", var_name="Portfolio", value_name="Total Return")

    # Multiply by 100 to convert to percentage
    df_melt["Total Return"] *= 100

    fig = px.line(
        df_melt,
        x="Date",
        y="Total Return",
        color="Portfolio",
        title=" ",
        template="simple_white"
    )

    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.05,
            xanchor="center",
            x=0.5
        )
    )
    
    # ✨ Custom hover menu
    fig.update_traces(
        mode="lines",
        hovertemplate=(
            "<b>%{fullData.name}</b><br>" +
            "Date: %{x|%b %Y}<br>" +  # Changed to show Month Year
            "Total Return: %{y:.2f}%<extra></extra>"
        )
    )

    fig.update_layout(**DEFAULT_LAYOUT)

    return fig

In [29]:
plot_total_returns_streamlit(final_results_totalreturn)

In [77]:
plot_return_distributions_streamlit(final_results)

In [12]:
final_results.keys()[1:]

Index(['lifestrategy40', 'lifestrategy20', 'lifestrategy0'], dtype='object')

In [18]:
final_results_totalreturn = final_results.copy()
years = 5

for portfolio in final_results_totalreturn.keys()[1:]:

    final_results_totalreturn[portfolio] = (final_results[portfolio] + 1)**years - 1

final_results_totalreturn

,Date,lifestrategy40,lifestrategy20,lifestrategy0
0,01/1985,0.390924,0.268824,0.146667
1,02/1985,0.305907,0.208951,0.111951
2,03/1985,0.273974,0.208021,0.142045
3,04/1985,0.281822,0.194876,0.107983
4,05/1985,0.309129,0.217531,0.125955
...,...,...,...,...
412,12/2019,0.285241,0.101702,-0.081832
413,01/2020,0.287259,0.089492,-0.108280
414,02/2020,0.341954,0.115220,-0.111469
415,03/2020,0.374079,0.118935,-0.136208


In [1]:
final_maxdrawdown = None

for portfolio_name in portfolios:

    portfolio = load_portfolio(portfolio_name)
    data, weights = prune_data_portfolio(portfolio)

    prices = compute_portfolio_prices(portfolio, portfolio_name)

    maxdrawdown = max_drawdown(prices)

    final_maxdrawdown = merge_results(final_maxdrawdown, maxdrawdown)

plot_max_drawdown(final_maxdrawdown)

NameError: name 'portfolios' is not defined

In [ ]:
(annualized_return + 1)**years

NameError: name 'total_return' is not defined